In [18]:
import pandas as pd
import os, json, joblib
from pathlib import Path
from palmerpenguins import load_penguins
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


# Primer Taller MLOPS 
Presentador por Jacobo & Javier

In [2]:
penguins_df_raw = load_penguins()
penguins_df_raw.sample(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
157,Gentoo,Biscoe,46.5,13.5,210.0,4550.0,female,2007
148,Adelie,Dream,36.0,17.8,195.0,3450.0,female,2009
111,Adelie,Biscoe,45.6,20.3,191.0,4600.0,male,2009
66,Adelie,Biscoe,35.5,16.2,195.0,3350.0,female,2008
83,Adelie,Torgersen,35.1,19.4,193.0,4200.0,male,2008
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
187,Gentoo,Biscoe,48.4,16.3,220.0,5400.0,male,2008
243,Gentoo,Biscoe,52.2,17.1,228.0,5400.0,male,2009
145,Adelie,Dream,39.0,18.7,185.0,3650.0,male,2009
296,Chinstrap,Dream,42.4,17.3,181.0,3600.0,female,2007


In [3]:
penguins_df_raw.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

## Dataset Primeras Impresiones
Dando un primer vistazo al dataset podemos identificar que tienes la siguientes features:
1. Numericas: bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g, year
2. Categóricas: island, sex
3. Target: species

Inicialmente proponemos el siguiente tratamiento:
1. Imputar numericas con median
2. Imputar categorias con most_frequent
3. OneHot a categoricas

Para el entrenamiento de diversos modelos, realizamos la siguiente propuesta inicialmente, que podria cambiar dependiendo de su performance, aunque como obtener el mejor performance no es el objetivo de este taller probablemente nos mantengamos con estos:
1. Logistic Regression
2. Random Forest
3. SVC
4. Gradient Boosting



Separamos las variables X y el target y

In [4]:
X = penguins_df_raw.drop("species", axis = 1) #datos de entrada
y = penguins_df_raw["species"] # variable a predecir

In [5]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [6]:
y.head()

0    Adelie
1    Adelie
2    Adelie
3    Adelie
4    Adelie
Name: species, dtype: str

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y, #usamos stratify porque species es multiclase (adelie, gentoo, chinstrap), y esto amnetiene la proporcion de clases en train y validation
    random_state=42
)

In [8]:
X_train.shape

(275, 7)

In [9]:
X_val.shape

(69, 7)

In [10]:
# Pipeline para variables numéricas
numeric_transformer = Pipeline( #un pipeline es simplemente una lista ordenada de pasos que se ejecutan uno tras otro
    steps=[
        ("imputer", SimpleImputer(strategy="median")) #aqui decimos para las variables numericas aplica un imputador que reemplace los valores faltantes con la mediana
    ]
)

#cuando entrenemos el modelo el pipeline:
    #1. calcula la mediana usando solo los datos de entrenamiento
    #2. guarda esa mediana internamente
    #3. cuando llegue un nuevo dato en produccion, usa esa misma mediana

# Pipeline para variables categóricas
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

# Aqui tenemos dos pasos
    #1. si sex es nan, lo reemplaza por el valor mas frecuente
    #2. convierte variables categoricas en columnas binarias | handle_unknown="ignore", si aparece una isla nueva que nuncca vimos la ignora


In [11]:
numeric_features = [
    "bill_length_mm",
    "bill_depth_mm",
    "flipper_length_mm",
    "body_mass_g",
    "year"
]

categorical_features = [
    "island",
    "sex"
]


In [12]:
preprocessor = ColumnTransformer( #este objeto aplica transformaciones diferentes a diferentes columnas automaticamente
    transformers=[
        ("num", numeric_transformer, numeric_features), # a las columnas numericas aplica numeric_transformer
        ("cat", categorical_transformer, categorical_features) # a las columnas categoricas aplica categorical_transformer
    ]
) 
#junta todo en una sola matriz para el modelo


In [20]:
models = { #diccionario donde la clave es el nombre del modelo y el valor es la instancia del clasificador, esto para poderlos entrenar en un loop y no tener que reescribir codigo
    "logreg": LogisticRegression(max_iter=1000),
    "rf": RandomForestClassifier(random_state=42),
    "svm": SVC(probability=True, random_state=42),
    "gb": GradientBoostingClassifier(random_state=42),
    "knn": KNeighborsClassifier(n_neighbors=5), # ESTE ES EL MODELO NUEVO PARA PROBAR QUE LOS VOLS QUEDARON BIEN, SI DESEA RECREAR ELIMINARLO DE /models y de registry.json y volver a correr
}

trained_models = {} # diccionario vacio para guardar los modelos ya entrenados

for name, clf in models.items(): #iterar sobre cada modelo, ej: primera iteracion -> name = "logreg", clf = LogisticRegression(max_iter = 1000)
    pipe = Pipeline(steps=[ #creamos un pipeline que hace 1. preprocesamiento y 2. clasificacion
        ("preprocessor", preprocessor),
        ("model", clf)
    ])
    # cuando llegue aqui, internamente el preprocessor aprende: 1. mediandas, moda, categorias para one-hot, 2. transforma los datos, 3. el modelo se entrena con datos transformados
    pipe.fit(X_train, y_train) # en resumen aqui ejecuta, 1. imputacion, 2. enconding, 3. transformacion, 4. entrenamiento del modelo 
    #IMPORTANTE: el preprocesamiento se ajusto SOLO con X_trian para evitar data leakage

    preds = pipe.predict(X_val) #el pipeline transforma automaticamente X_val, el modelo predice
    acc = accuracy_score(y_val, preds) #calculamos accuracy

    trained_models[name] = pipe #guardar el pipeline completo entrenado, no solo el clasificador sino preprocessor + modelo

    print(f"{name} -> val accuracy: {acc:.4f}")#mostrar resultados


/workspace/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logreg -> val accuracy: 1.0000
rf -> val accuracy: 1.0000
svm -> val accuracy: 0.7681
gb -> val accuracy: 1.0000
knn -> val accuracy: 0.7826


In [21]:
# Carpeta de modelos en el CONTENEDOR (volumen compartido)
MODELS_DIR = Path("/workspace/models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# 1. Guardar todos los pipelines entrenados como .joblib en el volumen compartido
for name, pipe in trained_models.items():
    output_path = MODELS_DIR / f"{name}.joblib"
    joblib.dump(pipe, output_path)
    print(f"Modelo '{name}' guardado en {output_path}")

# 2. Crear / actualizar registry.json en el volumen compartido
registry_path = MODELS_DIR / "registry.json"

# Si ya existe, lo cargamos y actualizamos; si no, lo creamos desde cero
if registry_path.exists():
    with open(registry_path, "r") as f:
        registry = json.load(f)
else:
    registry = {
        "default_model": None,
        "available_models": []
    }

available = set(registry.get("available_models", []))

for name in trained_models.keys():
    available.add(name)

registry["available_models"] = sorted(list(available))

# Si quieres que el modelo nuevo (knn) sea el default, déjalo así:
registry["default_model"] = "knn"
# Si prefieres mantener "rf" como default, cambia la línea anterior por:
# registry["default_model"] = "rf"

with open(registry_path, "w") as f:
    json.dump(registry, f, indent=2)

print("Modelos guardados en /workspace/models")
print("Registry actualizado:", registry)

Modelo 'logreg' guardado en /workspace/models/logreg.joblib
Modelo 'rf' guardado en /workspace/models/rf.joblib
Modelo 'svm' guardado en /workspace/models/svm.joblib
Modelo 'gb' guardado en /workspace/models/gb.joblib
Modelo 'knn' guardado en /workspace/models/knn.joblib
Modelos guardados en /workspace/models
Registry actualizado: {'default_model': 'knn', 'available_models': ['gb', 'knn', 'logreg', 'rf', 'svm']}
